# ऑटोजेनसह एजेंटिक RAG

हा नोटबुक ऑटोजेन एजंट्ससह पुनर्प्राप्ती-वर्धित जनरेशन (RAG) अंमलात आणण्याचे प्रदर्शन करतो, ज्यामध्ये सुधारित मूल्यमापन क्षमता आहेत.


SQLite आवृत्ती दुरुस्ती  
जर तुम्हाला खालील त्रुटी आली:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

तुमच्या नोटबुकच्या सुरुवातीला हा कोड ब्लॉक अनकॉमेंट करा:  


In [2]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
import os
import time
import asyncio
from typing import List, Dict
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient

import chromadb

load_dotenv()

True

## क्लायंट तयार करा

सर्वप्रथम, आपण Azure AI Chat Completion Client प्रारंभ करतो. हा क्लायंट वापरकर्त्याच्या प्रश्नांना उत्तर तयार करण्यासाठी Azure OpenAI सेवेसोबत संवाद साधण्यासाठी वापरला जाईल.


In [4]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

## व्हेक्टर डेटाबेस प्रारंभ

आम्ही ChromaDB कायमस्वरूपी स्टोरेजसह प्रारंभ करतो आणि सुधारित नमुना दस्तऐवज जोडतो. ChromaDB दस्तऐवज साठवण्यासाठी आणि पुनर्प्राप्त करण्यासाठी वापरला जाईल, जे अचूक प्रतिसाद निर्माण करण्यासाठी संदर्भ प्रदान करतात.


In [5]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.create_collection(
    name="travel_documents",
    metadata={"description": "travel_service"},
    get_or_create=True
)

# Enhanced sample documents
documents = [
    "Contoso Travel offers luxury vacation packages to exotic destinations worldwide.",
    "Our premium travel services include personalized itinerary planning and 24/7 concierge support.",
    "Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.",
    "Popular destinations include the Maldives, Swiss Alps, and African safaris.",
    "Contoso Travel provides exclusive access to boutique hotels and private guided tours."
]

# Add documents with metadata
collection.add(
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))],
    metadatas=[{"source": "training", "type": "explanation"} for _ in documents]
)

## संदर्भ प्रदाता अंमलबजावणी

`ContextProvider` वर्ग विविध स्रोतांकडून संदर्भ पुनर्प्राप्ती आणि एकत्रीकरण हाताळतो:

1. **व्हेक्टर डेटाबेस पुनर्प्राप्ती**: ट्रॅव्हल डॉक्युमेंट्सवर सिमॅंटिक सर्च करण्यासाठी ChromaDB चा वापर करतो
2. **हवामान माहिती**: प्रमुख शहरांसाठी सिम्युलेटेड हवामान डेटाबेस राखतो
3. **एकत्रित संदर्भ**: दस्तऐवज आणि हवामान डेटा एकत्र करून व्यापक संदर्भ तयार करतो

महत्त्वाच्या पद्धती:
- `get_retrieval_context()`: क्वेरीच्या आधारे संबंधित दस्तऐवज पुनर्प्राप्त करतो
- `get_weather_data()`: निर्दिष्ट ठिकाणांसाठी हवामान माहिती प्रदान करतो
- `get_unified_context()`: सुधारित प्रतिसादांसाठी दस्तऐवज आणि हवामान संदर्भ एकत्र करतो


In [6]:
class ContextProvider:
    def __init__(self, collection):
        self.collection = collection
        # Simulated weather database
        self.weather_database = {
            "new york": {"temperature": 72, "condition": "Partly Cloudy", "humidity": 65, "wind": "10 mph"},
            "london": {"temperature": 60, "condition": "Rainy", "humidity": 80, "wind": "15 mph"},
            "tokyo": {"temperature": 75, "condition": "Sunny", "humidity": 50, "wind": "5 mph"},
            "sydney": {"temperature": 80, "condition": "Clear", "humidity": 45, "wind": "12 mph"},
            "paris": {"temperature": 68, "condition": "Cloudy", "humidity": 70, "wind": "8 mph"},
        }
    
    def get_retrieval_context(self, query: str) -> str:
        """Retrieves relevant documents from vector database based on query."""
        results = self.collection.query(
            query_texts=[query],
            include=["documents", "metadatas"],
            n_results=2
        )
        context_strings = []
        if results and results.get("documents") and len(results["documents"][0]) > 0:
            for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
                context_strings.append(f"Document: {doc}\nMetadata: {meta}")
        return "\n\n".join(context_strings) if context_strings else "No relevant documents found"
    
    def get_weather_data(self, location: str) -> str:
        """Simulates retrieving weather data for a given location."""
        if not location:
            return ""
            
        location_key = location.lower()
        if location_key in self.weather_database:
            data = self.weather_database[location_key]
            return f"Weather for {location.title()}:\n" \
                   f"Temperature: {data['temperature']}°F\n" \
                   f"Condition: {data['condition']}\n" \
                   f"Humidity: {data['humidity']}%\n" \
                   f"Wind: {data['wind']}"
        else:
            return f"No weather data available for {location}."
    
    def get_unified_context(self, query: str, location: str = None) -> str:
        """Returns a unified context combining both document retrieval and weather data."""
        retrieval_context = self.get_retrieval_context(query)
        
        weather_context = ""
        if location:
            weather_context = self.get_weather_data(location)
            weather_intro = f"\nWeather Information for {location}:\n"
        else:
            weather_intro = ""
        
        return f"Retrieved Context:\n{retrieval_context}\n\n{weather_intro}{weather_context}"

## एजंट कॉन्फिगरेशन

आम्ही पुनर्प्राप्ती आणि सहाय्यक एजंट्स कॉन्फिगर करतो. पुनर्प्राप्ती एजंट सेमॅंटिक शोधाचा वापर करून संबंधित माहिती शोधण्यात विशेष तज्ज्ञ आहे, तर सहाय्यक पुनर्प्राप्त केलेल्या माहितीनुसार सविस्तर प्रतिसाद तयार करतो.


In [7]:
# Create agents with enhanced capabilities
assistant = AssistantAgent(
    name="assistant",
    model_client=client,
    system_message=(
        "You are a helpful AI assistant that provides answers using ONLY the provided context. "
        "Do NOT include any external information. Base your answer entirely on the context given below."
    ),
)

## RAG सह क्वेरी प्रक्रिया

आम्ही `ask_rag` फंक्शन परिभाषित करतो जे क्वेरी सहाय्यकाकडे पाठवते, प्रतिसाद प्रक्रिया करते आणि त्याचे मूल्यांकन करते. हे फंक्शन सहाय्यकाशी संवाद हाताळते आणि प्रतिसादाच्या गुणवत्तेचे मोजमाप करण्यासाठी मूल्यांकनकर्ता वापरते.


In [8]:
async def ask_rag_agent(query: str, context_provider: ContextProvider, location: str = None):
    """
    Sends a query to the assistant agent with context from the provider.
    
    Args:
        query: The user's question
        context_provider: The context provider instance
        location: Optional location for weather queries
    """
    try:
        # Get unified context
        context = context_provider.get_unified_context(query, location)
        
        # Augment the query with context
        augmented_query = (
            f"{context}\n\n"
            f"User Query: {query}\n\n"
            "Based ONLY on the above context, please provide a helpful answer."
        )

        # Send the augmented query to the assistant
        start_time = time.time()
        response = await assistant.on_messages(
            [TextMessage(content=augmented_query, source="user")],
            cancellation_token=CancellationToken(),
        )
        processing_time = time.time() - start_time
        
        return {
            'query': query,
            'response': response.chat_message.content,
            'processing_time': processing_time,
            'location': location
        }
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# उदाहरण वापर

आम्ही evaluator प्रारंभ करतो आणि ज्या क्वेरी प्रक्रिया करायच्या आणि मूल्यांकन करायच्या आहेत त्या परिभाषित करतो.


In [ ]:
async def main():
    # Initialize context provider
    context_provider = ContextProvider(collection)
    
    # Example queries
    queries = [
        {"query": "What does Contoso's travel insurance cover?"},
        {"query": "What's the weather like in London?", "location": "london"},
        {"query": "What luxury destinations does Contoso offer and what's the weather in Paris?", "location": "paris"},
    ]
    
    print("=== Autogen RAG Demo ===")
    for query_data in queries:
        query = query_data["query"]
        location = query_data.get("location")
        
        print(f"\n\nQuery: {query}")
        if location:
            print(f"Location: {location}")
        
        # Show the context being used
        context = context_provider.get_unified_context(query, location)
        print("\n--- Context Used ---")
        print(context)
        print("-------------------")
        
        # Get response from the agent
        result = await ask_rag_agent(query, context_provider, location)
        if result:
            print(f"\nResponse: {result['response']}")
        print("\n" + "="*50)

## स्क्रिप्ट चालवा

आम्ही तपासतो की स्क्रिप्ट परस्परसंवादी वातावरणात चालू आहे की सामान्य स्क्रिप्टमध्ये, आणि त्यानुसार मुख्य फंक्शन चालवतो.


In [12]:
if __name__ == "__main__":
    if asyncio.get_event_loop().is_running():
        await main()
    else:
        asyncio.run(main())

=== Autogen RAG Demo ===


Query: What does Contoso's travel insurance cover?

--- Context Used ---
Retrieved Context:
Document: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


-------------------

Response: Contoso's travel insurance covers medical emergencies, trip cancellations, and lost baggage.



Query: What's the weather like in London?
Location: london

--- Context Used ---
Retrieved Context:
Document: Popular destinations include the Maldives, Swiss Alps, and African safaris.
Metadata: {'source': 'training', 'type': 'explanation'}

Document: Contoso Travel offers luxury vacation packages to exotic destinations worldwide.
Metadata: {'source': 'training', 'type': 'explanation'}


Weather Information for london:
Weather for London:
T


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) चा वापर करून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी, व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
